# Importing Data

In [21]:
# import libraries
from bs4 import BeautifulSoup
from urllib import request
import time
import random
import numpy as np
import pandas as pd
import re
import csv

ModuleNotFoundError: No module named 'unicode'

In this file, I will make the previous web scraping codes into a pipelin.

1. functionget_hyperlinks will created a csv file with the name: component_name_hyperlinks.csv. This file stores all the hyperlinks of each item in the search page of url from newegg website.

   a. component_name is the name of the computer components i.e intel_cpu
   
   b. The url should be in the following form url = "https://www.newegg.com/p/pl?N=100007671%20601306860&Page={}"
   
   c. num_pages is the total number of pages to be web scraped in the searching page of the url at newegg website
   
2. 

## Web Scraping all Hyperlinks

In [ ]:
def get_hyperlinks(component_name,url,num_pages):
    ## create a csv table to store all the hyperlinks of CPU
    filename = "{}_hyperlinks.csv".format(component_name)
    f = open(filename, "w", encoding='utf-8')
    headers = "price,product_detail,hyperlink\n"
    f.write(headers)
    
    
    # Scraping num_pages of my_url from newegg
    print('Scraping {} searching pages:'.format(component_name))
    
    for i in range(1, num_pages+1):
        my_url = url.format(i)
        uClient = request.urlopen(my_url)
        page_html = uClient.read()
        uClient.close()
        page_soup = BeautifulSoup(page_html, "html.parser")
        containers = page_soup.findAll("div", {"class": "item-container"})

        for container in containers:
            ## Find hyperlink that directs to the webpage of that particular product
            hyperlink = list(container.children)[3]['href']
            ## discard data with bad format(will be revised in the future)
            if not hyperlink:
                continue
            
            ## find the current price of this item
            Pli=container.find('li',{"class":"price-current"})

            strong=Pli.find('strong')
            sup=Pli.find('sup')
            
            strong=strong.text if strong else ''               
            sup=sup.text if strong else ''

            price=strong+sup
            if price:
                price=price.replace(',','')
            else:
                continue
                
            ## find the product detail of the product
            product_detail = list(container.children)[3].img["alt"].replace(",", "  ")
            
            
            f.write(price+","+product_detail+","+hyperlink+"\n")

            time.sleep(random.random()+0.5)
        time.sleep(10+random.random())
        print(i)
    
    f.close()

## Get all specs from the hyperlink_file 

In [22]:
def specs_to_csv(Specs,component_name):
## find the current header of the file if none set header to an empty list
        with open(component_name+"_specs.csv", "r") as g:
            reader = csv.reader(g)
            try:
                header=next(reader)
            except:
                header=list(Specs.keys())
                with open(component_name+"_specs.csv", "a") as f:
                    f.write(','.join(header))
        
        ## check whether current header is the same as the Specs.keys()
        for col in Specs.keys():
            if col not in header:
            
                ## use pandas to add a column with 'None' to the right
                df = pd.read_csv(component_name+"_specs.csv")
                new_column = pd.DataFrame({col: ['None' for i in range(len(df))]})
                df = df.merge(new_column, left_index = True, right_index = True)
                df.to_csv(component_name+"_specs.csv", index = False)
                   
                header.append(col)
        
        ## write the current Specs to the csv file
        Specs_str=''
        for col in header:
            Specs_str=Specs_str+str(Specs.get(col,'None')).replace(',',' ')+','
        Specs_str=Specs_str+'\n'
            
        with open(component_name+"_specs.csv", "a") as f:
            f.write(Specs_str)

In [10]:
## return a list of specs by webscraping all the hyperlinks in the 
#  component_name+"_hyperlinks.csv" file created by get_hyperlinks function 
#  start is the index of the starting hyperlink
def write_specs(start,component_name):
    hyperlinks=pd.read_csv(component_name+"_hyperlinks.csv")
    
    ## create a csv table to store all the specs of component if there is 
    ## no such file
    filename = component_name+"_specs.csv"
    with open(filename, "a")  as f:
        pass
    
    
    
    ## all cpu specs will be stored in the list total_specs
    ## then total_specs will be stored in a csv table

    i=0
    print('Scraping each item page of {} for specs:'.format(component_name))
    
    # len(hyperlinks['hyperlink'])
    for i in range(start,start+1):
        url=hyperlinks['hyperlink'][i]
        ## if the url cannot be open, continue
        try:
            uClient = request.urlopen(url)
            page_html = uClient.read()
            uClient.close()
            page_soup = BeautifulSoup(page_html, "html.parser")
        except:
            continue
            
        ## create a dict Specs to store the specification information of the item
        try:
            Specs={'price':hyperlinks['price'][i]}
            
            specs = page_soup.find("div", {"id": "Specs"})
            fieldsets=specs.findAll("fieldset")
                 
            for fieldset in fieldsets:
                dls=fieldset.findAll("dl")
                for dl in dls:
                    Specs[dl.find("dt").text]=dl.find("dd").text          
        except:
            continue
            
        specs_to_csv(Specs,component_name)
               
        
        i+=1
        print(i)

        time.sleep(10+random.random())


In [23]:
write_specs(1,'AMD_MB')

Scraping each item page of AMD_MB for specs:


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd7 in position 1: invalid continuation byte

In [15]:
Specs_str='299.99,ASUS,ROG,STRIX X570-E GAMING,AM4,AMD AM4 Socket 3rd and 2nd AMD Ryzen / 2nd and 1st Gen AMD Ryzen with Radeon Vega Graphics Processors* Refer to www.asus.com for CPU support list,AMD X570,4×288pin,3rd Gen AMD Ryzen ProcessorsDDR4 4400(O.C.)/ 4266(O.C.)/ 4133(O.C.)/ 4000(O.C.)/ 3866(O.C.)/ 3600(O.C.)/ 3400(O.C.)/ 3200(O.C.)/ 3000(O.C.)/ 2800(O.C.)/ 2666/ 2400/ 2133 MHz Un-buffered Memory *2nd Gen AMD Ryzen ProcessorsDDR4 3600(O.C.)/ 3400(O.C.)/ 3200(O.C.)/ 3000(O.C.)/ 2800(O.C.)/ 2666/ 2400/ 2133 MHz Un-buffered Memory *2nd and 1st Gen AMD Ryzen with Radeon Vega Graphics ProcessorsDDR4 3200(O.C.)/ 3000(O.C.)/ 2800(O.C.)/ 2666/ 2400/ 2133 MHz Un-buffered Memory* Refer to www.asus.com for the Memory QVL (Qualified Vendors Lists).,128GB,Dual Channel,3rd Gen AMD Ryzen Processors2 x PCIe 4.0 x16 (x16 or dual x8)2nd Gen AMD Ryzen Processors2 x PCIe 3.0 x16 (x16 or dual x8)2nd and 1st Gen AMD Ryzen with Radeon Vega Graphics Processors1 x PCIe 3.0 x16 (x8 mode)AMD X570 chipset1 x PCIe 4.0 x16 (max at x4 mode) ** PCIeX16_3 slot shares bandwidth with PCIeX1_2.,2 x PCIe 4.0 x1,8 x SATA 6Gb/s,3rd Gen AMD Ryzen Processors:1 x M.2 Socket 3  with M Key  Type 2242/2260/2280/22110 (PCIE 4.0 x4 and SATA modes) storage devices support2nd Gen AMD Ryzen / 2nd and 1st Gen AMD Ryzen with Radeon Vega Graphics Processors:1 x M.2 Socket 3  with M Key  Type 2242/2260/2280/22110 (PCIE 3.0 x4 and SATA modes) storage devices supportAMD X570 chipset:1 x M.2 Socket 3  with M Key  Type 2242/2260/2280/22110 (PCIE 4.0 x4 and SATA modes) storage devices support,0/1/10,Integrated Graphics in the 2nd and 1st Gen AMD Ryzen with Radeon Vega Graphics ProcessorsMulti-VGA output support: HDMI/DisplayPort ports- Supports HDMI 2.0b with maximum resolution of 4096 x 2160 @ 60 Hz- Supports DisplayPort 1.2 with max. resolution 4096 x 2160 @ 60 Hz,SupremeFX High Definition Audio CODEC S1220A,8 Channels* Due to limitations in HDA bandwidth  32-Bit / 192 kHz is not supported for 8-Channel audio.,Realtek RTL8125-CG,Intel I211-AT,10/100/1000/2500Mbps + 10/100/1000Mbps,Supports channel bandwidth: HT20/HT40/HT80/HT160Intel Wi-Fi 6 AX2002 x 2 Wi-Fi 6 (802.11 a/b/g/n/ac/ax) with MU-MIMO supports dual frequency band 2.4/5 GHzUp to 2.4Gbps transfer speed,Bluetooth 5.0,1 x DisplayPort1 x HDMI1 x Optical S/PDIF out5 x Audio jack(s)1 x Anti-surge 2.5G LAN (RJ45) port1 x USB BIOS Flashback Button(s)1 x ASUS Wi-Fi Module7 x USB 3.2 Gen 2 (up to 10Gbps) ports (Type-A)1 x USB 3.2 Gen 2 (up to 10Gbps) ports (Type-C)Anti-surge LAN (RJ45) port,1 x USB 3.2 Gen 2 (up to 10Gbps) connector1 x USB 3.2 Gen 1 (up to 5Gbps) connector support additional 2 USB ports2 x USB 2.0 connector(s) support(s) additional 4 USB 2.0 port(s),1 x PCH_FAN1 x AAFP connector2 x Aura RGB Strip Header(s)2 x Addressable Gen 2 header(s)1 x SPI TPM header1 x CPU Fan connector(s)1 x CPU OPT Fan connector(s)2 x Chassis Fan connector(s)1 x AIO_PUMP connector1 x W_PUMP+ connector1 x Front panel audio connector(s) (AAFP)1 x Thermal sensor connector(s)1 x Clear CMOS jumper(s)1 x Node Connector(s)1 x System panel connector1 x T_Sensor Connector1 x M.2 Fan Header,ATX,RGB,12.0" x 9.6",1 x 24-pin EATX Power connector(s)1 x 8-pin ATX 12V Power connector(s)1 x 4-pin ATX 12V Power connector(s),Multi-GPU Support:3rd and 2nd Gen AMD Ryzen ProcessorsSupports NVIDIA 2-Way SLI TechnologySupports AMD 3-Way CrossFireX Technology2nd and 1st Gen AMD Ryzen with Radeon Vega Graphics ProcessorsSupports AMD 2-Way CrossFireX TechnologyROG Exclusive Features:ROG Exclusive Software- RAMCache III- CPU-Z- GameFirst V- Sonic Studio III + Sonic Studio Link- Sonic Radar III- OverwolfSpecial Features:5-Way Optimization by Dual Intelligent Processors 5- Whole system optimization with a single click! 5-Way Optimization tuning key perfectly consolidates TPU  EPU  DIGI+ VRM  Fan Xpert 4  and Turbo App together  providing better CPU performance  efficient power saving  precise digital power control  whole system cooling and even tailor your own app usagesTPU- Auto Tuning  TurboV  GPU BoostGamer\'s Guardian:- DRAM Overcurrent Protection- Stainless Steel Back I/O- Highly Durable Components- DIGI+ VRM- SafeSlot- ESD Guards on LAN  Audio  and USB portsASUS EPU:- EPUAURA:- Aura Lighting Control- Addressable Gen 2 HeaderASUS Exclusive Features:- USB BIOS Flashback- AI Suite 3- Ai Charger- ASUS NODE: hardware control interface- BIOS Flashback ButtonASUS EZ DIY:- ASUS CrashFree BIOS 3- ASUS EZ Flash 3ASUS Q-Design:- ASUS Q-Code- ASUS Q-LED (CPU  DRAM  VGA  Boot Device LED)- ASUS Q-Slot- ASUS Q-DIMMGaming Aesthetics:- AURA-RGB LightingBIOS:256MB Flash ROM  UEFI AMI BIOS  PnP  WfM2.0  SM BIOS 3.2  ACPI 6.2Manageability:WOL  PXESupport Disc:OverwolfAnti-virus software (OEM version)WinRAROperating System:Windows 10 64-bit,Accessories:User\'s manual4 x SATA 6Gb/s cable(s)1 x M.2 Screw Package1 x Supporting DVD1 x Strix door hanger1 x ROG Strix stickers1 x Cable ties pack(s)1 x Wi-Fi Antenna(s)1 x Extension Cable for RGB strips (80cm)1 x Extension cable for Addressable LED1 x Thermistor cable(s)1 x ROG Thank you card,July 07  2019,\n'

In [17]:
with open('AMD_MB'+"_specs.csv", "a") as f:
    f.write(Specs_str)

In [ ]:
## Here is the total pipeline
def pipeline(component_name,url,num_pages):
    get_hyperlinks(component_name,url,num_pages)
    total_specs=get_specs(component_name)
    write_specs(total_specs,component_name)

In [ ]:
url='https://www.newegg.com/AMD-Motherboards/SubCategory/ID-22/Page-{}'
component_name='AMD_MB'
num_pages=52 
pipeline(component_name,url,num_pages)